In [1]:
import pandas as pd
import matplotlib.pyplot as plt
# import wbgapi as wb
# from scipy.stats import linregress

## Extract Data
### Create reference data:

#### Countries allowed:
* Number of Gold Medals >0 & Rank by Total Medals <=20

#### Medal types:
* Gold & Total

In [44]:
# Load extracted via sql athena data about medals
medals_path = "Resources/athena_query_medals.csv"
medals = pd.read_csv(medals_path)
medals

,noc,gold,total,rank,rank by total
0,United States of America,39,113,1,1
1,People's Republic of China,38,88,2,2
2,Japan,27,58,3,5
3,Great Britain,22,65,4,4
4,ROC,20,71,5,3
5,Australia,17,46,6,6
6,Netherlands,10,36,7,9
7,France,10,33,8,10
8,Germany,10,37,9,8
9,Italy,10,40,10,7


In [45]:
noc = medals["noc"].values
noc

array(['United States of America', "People's Republic of China", 'Japan',
       'Great Britain', 'ROC', 'Australia', 'Netherlands', 'France',
       'Germany', 'Italy', 'Canada', 'Brazil', 'New Zealand', 'Cuba',
       'Hungary', 'Republic of Korea', 'Poland', 'Spain', 'Switzerland',
       'Turkey', 'Ukraine'], dtype=object)

In [63]:
# Export cleaned data to csv
medals.to_csv("../Output/medals.csv", index = False)

In [68]:
# Load extracted via sql athena data about athletes - that correspond to 
athl_path = "Resources/athena_query_athletes.csv"
athletes = pd.read_csv(athl_path)
athletes

,name,noc,discipline
0,ABAD Nestor,Spain,Artistic Gymnastics
1,ABAGNALE Giovanni,Italy,Rowing
2,ABALDE Alberto,Spain,Basketball
3,ABALDE Tamara,Spain,Basketball
4,ABALO Luc,France,Handball
...,...,...,...
6354,RIEDEMANN Laura,Germany,Swimming
6355,RIEDER Celine,Germany,Swimming
6356,RIENDEAU Vincent,Canada,Diving
6357,RIERA ZUZUARREGUI Lola,Spain,Hockey


In [69]:
# Apply BR: Each cell should contain a single value

# Using Pandas split Name column values into two separated columns
columnsplit = athletes['name'].str.split(" ", expand=True)
athletes = athletes.assign(last_name=columnsplit[0],first_name=columnsplit[1])
athletes

,name,noc,discipline,last_name,first_name
0,ABAD Nestor,Spain,Artistic Gymnastics,ABAD,Nestor
1,ABAGNALE Giovanni,Italy,Rowing,ABAGNALE,Giovanni
2,ABALDE Alberto,Spain,Basketball,ABALDE,Alberto
3,ABALDE Tamara,Spain,Basketball,ABALDE,Tamara
4,ABALO Luc,France,Handball,ABALO,Luc
...,...,...,...,...,...
6354,RIEDEMANN Laura,Germany,Swimming,RIEDEMANN,Laura
6355,RIEDER Celine,Germany,Swimming,RIEDER,Celine
6356,RIENDEAU Vincent,Canada,Diving,RIENDEAU,Vincent
6357,RIERA ZUZUARREGUI Lola,Spain,Hockey,RIERA,ZUZUARREGUI


In [70]:
# Drop Name column and Reorder columns
athletes = athletes.drop(['name'], axis=1)
athletes = athletes[["last_name","first_name", "noc", "discipline"]]
athletes.head()

,last_name,first_name,noc,discipline
0,ABAD,Nestor,Spain,Artistic Gymnastics
1,ABAGNALE,Giovanni,Italy,Rowing
2,ABALDE,Alberto,Spain,Basketball
3,ABALDE,Tamara,Spain,Basketball
4,ABALO,Luc,France,Handball


In [71]:
# Apply BR: Values in cells should be Capitalized
athletes['last_name'] = athletes['last_name'].str.capitalize()
athletes['first_name'] = athletes['first_name'].str.capitalize()
athletes

,last_name,first_name,noc,discipline
0,Abad,Nestor,Spain,Artistic Gymnastics
1,Abagnale,Giovanni,Italy,Rowing
2,Abalde,Alberto,Spain,Basketball
3,Abalde,Tamara,Spain,Basketball
4,Abalo,Luc,France,Handball
...,...,...,...,...
6354,Riedemann,Laura,Germany,Swimming
6355,Rieder,Celine,Germany,Swimming
6356,Riendeau,Vincent,Canada,Diving
6357,Riera,Zuzuarregui,Spain,Hockey


In [72]:
# Export cleaned data to csv
athletes.to_csv("../Output/athletes.csv")

In [54]:
# Load extracted via sql athena data about teams with business rules applied:
# allowed countries and allowed disciplines
team_path = "Resources/athena_query_teams.csv"
teams = pd.read_csv(team_path)
teams

,name,discipline,noc,event
0,Australia,Archery,Australia,Men's Team
1,Australia,Archery,Australia,Mixed Team
2,Brazil,Archery,Brazil,Mixed Team
3,Canada,Archery,Canada,Mixed Team
4,China,Archery,People's Republic of China,Men's Team
...,...,...,...,...
408,ROC,Water Polo,ROC,Women
409,Spain,Water Polo,Spain,Men
410,Spain,Water Polo,Spain,Women
411,United States,Water Polo,United States of America,Men


In [65]:
# Export cleaned data to csv
teams.to_csv("../Output/teams.csv")

In [60]:
# Load extracted via sql athena data about coaches with business rules applied:
# allowed countries and allowed disciplines
coach_path = "Resources/athena_query_coaches.csv"
coaches = pd.read_csv(coach_path)
coaches

,name,noc,discipline,event
0,AIKMAN Siegfried Gottlieb,Japan,Hockey,Men
1,AL SAADI Kais,Germany,Hockey,Men
2,ANNAN Alyson,Netherlands,Hockey,Women
3,ARNAU CREUS Xavier,Japan,Hockey,Women
4,BASSO Paola,Great Britain,Artistic Swimming,Duet
...,...,...,...,...
98,WANG Jie,People's Republic of China,Artistic Swimming,Team
99,WANG Yang,People's Republic of China,Hockey,Women
100,ZAITSEVA Olesia,Ukraine,Artistic Swimming,Duet
101,ZHANG Xiaohuan,People's Republic of China,Artistic Swimming,NaN


In [61]:
# Apply BR: Each cell should contain a single value

# Using Pandas split Name column values into two separated columns
columnsplit = coaches['name'].str.split(" ", expand=True)
coaches = coaches.assign(last_name=columnsplit[0],first_name=columnsplit[1])
# Drop Name column and Reorder columns
coaches = coaches.drop(['name'], axis=1)
coaches = coaches[["last_name","first_name", "noc", "discipline", "event"]]
# Apply BR: Values in cells should be Capitalized
coaches['last_name'] = coaches['last_name'].str.capitalize()
coaches['first_name'] = coaches['first_name'].str.capitalize()
coaches

,last_name,first_name,noc,discipline,event
0,Aikman,Siegfried,Japan,Hockey,Men
1,Al,Saadi,Germany,Hockey,Men
2,Annan,Alyson,Netherlands,Hockey,Women
3,Arnau,Creus,Japan,Hockey,Women
4,Basso,Paola,Great Britain,Artistic Swimming,Duet
...,...,...,...,...,...
98,Wang,Jie,People's Republic of China,Artistic Swimming,Team
99,Wang,Yang,People's Republic of China,Hockey,Women
100,Zaitseva,Olesia,Ukraine,Artistic Swimming,Duet
101,Zhang,Xiaohuan,People's Republic of China,Artistic Swimming,NaN


In [66]:
# Export cleaned data to csv
coaches.to_csv("../Output/coaches.csv")

### Join tables coaches and teams

In [80]:
# Add coaches to teams table, join on noc, discipline and event
coach_team = pd.merge(teams, coaches, on=["noc","discipline","event"], how = 'left')
coach_team

,name,discipline,noc,event,last_name,first_name
0,Australia,Archery,Australia,Men's Team,NaN,NaN
1,Australia,Archery,Australia,Mixed Team,NaN,NaN
2,Brazil,Archery,Brazil,Mixed Team,NaN,NaN
3,Canada,Archery,Canada,Mixed Team,NaN,NaN
4,China,Archery,People's Republic of China,Men's Team,NaN,NaN
...,...,...,...,...,...,...
422,ROC,Water Polo,ROC,Women,Gaidukov,Aleksandr
423,Spain,Water Polo,Spain,Men,Martin,Lozano
424,Spain,Water Polo,Spain,Women,Oca,Gaia
425,United States,Water Polo,United States of America,Men,Udovicic,Dejan


* as we can see from the result of join, coaches data set does not have enough information about coaches, thus we have much more teams than coaches. If we do the join and save it as a separate table into our database it will either reduce information about teams (if we do inner join), or have a lot of NaN values if we do left join. So we will create two separte table in our final database - teams and coaches. Thus, they will be ready for further analysis 

In [81]:
# test cell
coach_team.loc[coach_team["discipline"]=='Cycling BMX Freestyle']

,name,discipline,noc,event,last_name,first_name


In [79]:
# Load data set for entries_gender with business rules applied:
# allowed countries and allowed disciplines
gender_path = "../Resources/EntriesGender.csv"
genders = pd.read_csv(gender_path)
genders

,Discipline,Female,Male,Total
0,3x3 Basketball,32,32,64
1,Archery,64,64,128
2,Artistic Gymnastics,98,98,196
3,Artistic Swimming,105,0,105
4,Athletics,969,1072,2041
5,Badminton,86,87,173
6,Baseball/Softball,90,144,234
7,Basketball,144,144,288
8,Beach Volleyball,48,48,96
9,Boxing,102,187,289


In [11]:

# Check on one discpiline first
disciplines = all_team['Discipline'].unique()
disciplines

array(['3x3 Basketball', 'Archery', 'Artistic Gymnastics',
       'Artistic Swimming', 'Athletics', 'Baseball/Softball',
       'Basketball', 'Beach Volleyball', 'Cycling Track', 'Fencing',
       'Football', 'Handball', 'Hockey', 'Rhythmic Gymnastics',
       'Rugby Sevens', 'Swimming', 'Table Tennis', 'Triathlon',
       'Volleyball', 'Water Polo'], dtype=object)

In [16]:
a1 = all_athl.loc[all_athl["Discipline"]=="Table Tennis"]
a1

,Last_name,First_name,NOC,Discipline
20,Abdelaziz,Farah,Egypt,Table Tennis
75,Achanta,Kamal,India,Table Tennis
112,Afanador,Brian,Puerto Rico,Table Tennis
208,Alamian,Nima,Islamic Republic of Iran,Table Tennis
270,Alkhadrawi,Ali,Saudi Arabia,Table Tennis
...,...,...,...,...
10900,Yuan,Jia,France,Table Tennis
10959,Zaza,Hend,Syrian Arab Republic,Table Tennis
11004,Zhang,Lily,United States of America,Table Tennis
11010,Zhang,Mo,Canada,Table Tennis


In [17]:
t1 = all_team.loc[all_team["Discipline"]=="Table Tennis"]
t1

,Name,Discipline,NOC,Event
631,Australia,Table Tennis,Australia,Men's Team
632,HU Heming / TAPPER Melissa,Table Tennis,Australia,Mixed Doubles
633,Australia,Table Tennis,Australia,Women's Team
634,FEGERL Stefan / POLCANOVA Sofia,Table Tennis,Austria,Mixed Doubles
635,Austria,Table Tennis,Austria,Women's Team
636,Brazil,Table Tennis,Brazil,Men's Team
637,Brazil,Table Tennis,Brazil,Women's Team
638,WANG Zhen / ZHANG Mo,Table Tennis,Canada,Mixed Doubles
639,China,Table Tennis,People's Republic of China,Men's Team
640,XU Xin / LIU Shiwen,Table Tennis,People's Republic of China,Mixed Doubles


In [18]:
# Join athletes and teams to know which athletes teams
athl_team = pd.merge(a1, t1, on=["NOC","Discipline"])
athl_team

,Last_name,First_name,NOC,Discipline,Name,Event
0,Abdelaziz,Farah,Egypt,Table Tennis,Egypt,Men's Team
1,Abdelaziz,Farah,Egypt,Table Tennis,ASSAR Omar / MESHREF Dina,Mixed Doubles
2,Abdelaziz,Farah,Egypt,Table Tennis,Egypt,Women's Team
3,Assar,Omar,Egypt,Table Tennis,Egypt,Men's Team
4,Assar,Omar,Egypt,Table Tennis,ASSAR Omar / MESHREF Dina,Mixed Doubles
...,...,...,...,...,...,...
243,Pergel,Szandra,Hungary,Table Tennis,Hungary,Women's Team
244,Pota,Georgina,Hungary,Table Tennis,SZUDI Adam / PERGEL Szandra,Mixed Doubles
245,Pota,Georgina,Hungary,Table Tennis,Hungary,Women's Team
246,Szudi,Adam,Hungary,Table Tennis,SZUDI Adam / PERGEL Szandra,Mixed Doubles
